This notebook connects a simple grading applet to a google spreadsheet backend using the gspread library.

The spreadsheet has student names in the first column and homework assignments in the first row.

Using tab and shift-tab allows you to move between the fields which is efficient.

In [1]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('oauth-keyfile.json', scopes=scope)
gc = gspread.authorize(credentials)

In [2]:
grades = gc.open("202-gspread-test")
gradesheet = grades.worksheet('homework')

# get list of assignments from first row and column and dump blank value
assignments = gradesheet.row_values(1)[1:]
names = gradesheet.col_values(1)[1:]

In [4]:
from ipywidgets import widgets
from IPython.display import display

def print_list(change):
    # return a comma separated string with matches
    reduced_options = [f for f in assignments if assignment_input.value.lower() in f.lower()]
    assignment_matches.value = ', '.join(reduced_options)

def print_names(change):
    # return a comma separated string with matches
    reduced_options = [f for f in names if name_input.value.lower() in f.lower()]
    name_matches.value = ', '.join(reduced_options)

def write_to_gspread(change):
    # get row and col associated with matched name and assignment and write grade to cell
    thisrow = gradesheet.find(name_matches.value).row
    thiscol = gradesheet.find(assignment_matches.value).col
    gradesheet.update_cell(thisrow, thiscol, input_text_grade.value)
    
# define widgets
assignment_input = widgets.Text(description='assignment')
assignment_matches = widgets.HTML(description='assignment match')
name_input = widgets.Text(description='student')
name_matches = widgets.HTML(description='student match')
input_text_grade = widgets.Text(description='grade')
write = widgets.Button(description='write')
frame = widgets.HTML()

# set up callbacks
assignment_input.observe(print_list, names='value')
name_input.observe(print_names, names='value')
write.on_click(write_to_gspread)

# display widgets
display(widgets.HBox(children=(assignment_input, assignment_matches)))
display(input_text_grade)
display(widgets.HBox(children=(name_input, name_matches)))
display(write)
display(frame)